In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow as tf
from scipy.io import loadmat, savemat
import os
# from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import scipy.io as sio
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


dense_layers = [0, 1, 2, 3]
dense_sizes = [32, 64, 128, 256, 512]
conv_layers = [1, 2, 3, 4]
num_filters = [32, 64, 128, 256]
kernel_sizes = [2000, 2250, 2500, 2750, 3000]

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

2022-01-29 00:03:05.123008: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-01-29 00:03:05.123029: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
basePath = '/home/hymc/[0]Github/data'+ '/random_single source_100snapshots_1000sample/-1dB/train/train_data.mat'
basePath
x_train = sio.loadmat(basePath)
x_train = x_train['train_data']

In [4]:
# Input train noisy dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files(basePath+'/*/train/train_data.*', shuffle=False)
# for i in dataset.take(1).as_numpy_iterator():
#     print(i)

2022-01-29 00:03:16.795700: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-01-29 00:03:16.820206: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-29 00:03:16.820235: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: hymc-System-Product-Name
2022-01-29 00:03:16.820240: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: hymc-System-Product-Name
2022-01-29 00:03:16.820296: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 465.19.1
2022-01-29 00:03:16.820316: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 465.19.1
2022-01-29 00:03:16.820320: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 465.19.1
2022-01-29 00:03:16.820628: I tens

In [ ]:
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['train_data']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 

data_train = data_set[0]
for i in data_set:
    data_train = np.vstack((data_train,i)) # stack all the datasets vertically
    
data_train = data_train[5000:] # the first dataset is included in twice
data_train.shape

In [ ]:
# Input train noiseless dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files(basePath + '/*/train/train_origin_data.*', shuffle=False)
# for i in dataset.as_numpy_iterator():
#     print(i)
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['train_origin_data']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 

data_origin = data_set[0]
for i in data_set:
    data_origin = np.vstack((data_origin,i)) # stack all the datasets vertically
    
data_origin = data_origin[5000:] # the first dataset is included in twice
data_origin.shape

In [ ]:
# train data shuffle
np.random.seed(2020)
index = np.arange(data_train.shape[0])
print(index)
np.random.shuffle(index)
data_train = data_train[index]
data_origin = data_origin[index]

In [ ]:
# rootPath = os.getcwd()
# Windoes path
filePath = 'c:\\Users\\HYPC300\\OneDrive - 한양대학교\\GitHub\\Deeplearning-Autoencoder-DOA\\data';
#  MAC path:
# filePath = '/Users/button/Deeplearning-Autoencoder-DOA/data'
savePath = filePath + '\\result_lab1' + basePath

if not os.path.isdir(savePath):
    os.makedirs(savePath)
print(savePath)

In [ ]:
# Model configuration
nSample = x_train.shape[0]
epoch = 100
batch_size = 256
validation_split = 0.2

save_dir = os.path.join(savePath, 'save_models')
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
print(save_dir)

In [ ]:
class DOA_Denoising(Model):
    def __init__(self):
        super(DOA_Denoising, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(300, activation='relu'),
            # layers.Dropout(0.2),
            layers.Dense(200, activation='relu'),
            # layers.Dropout(0.2),
            layers.Dense(100, activation='relu')]) # * 300 200 100
        
        self.decoder = tf.keras.Sequential([
            layers.Dense(200, activation='relu'), #* 200 300 400
            # layers.Dropout(0.2),
            layers.Dense(300, activation='relu'),
            # layers.Dropout(0.2),
            layers.Dense(400, activation='linear')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
autoencoder = DOA_Denoising() 

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir logs

In [ ]:
model_name = (
    f"epoch-{epoch}-batch-{batch_size}-dense-300-200-100-time-{int(time.time())}"
)
print(model_name)

file_path = os.path.join(save_dir, model_name)
print(file_path)

# callbacks_list
callbacks_list = [
    EarlyStopping(monitor="val_loss", mode="min", verbose=0, patience=30),
    ModelCheckpoint(
        filepath=file_path,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="min",
    ),
    # TensorBoard(
    #     log_dir="./logs/{}".format(model_name),
    #     histogram_freq=1,
    #     write_graph=False,
    #     write_images=False,
    # ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1,
        patience=10,
        verbose=0,
        mode="min",
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
    ),
]

autoencoder.compile(optimizer="adam", loss="mse")

In [ ]:
history = autoencoder.fit(
    # dataset_final.take(num_train),
    data_train,
    data_origin,
    epochs=epoch,
    batch_size=batch_size,
    shuffle=True,
    # validation_data = dataset_final.skip(205000*0.8),
    validation_split=validation_split,
    callbacks=callbacks_list,  #! Do not put [] around callbacks_list variable
)

autoencoder.encoder.summary()
autoencoder.decoder.summary()

In [ ]:
# plt.plot(history.history["loss"], label="Traing Loss")
# plt.plot(history.history["val_loss"], label="Validation Loss")
# plt.legend()
# plt.savefig('10.png')
# plt.savefig('SNR_-30_training_loss.png')

In [ ]:
# Input test noisy dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files(basePath + '*/test/test_data.*', shuffle=False)
for i in dataset.as_numpy_iterator():
    print(i)
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['test_data']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 
data_test = data_set[0]
for i in data_set:
    data_test = np.vstack((data_test,i)) # stack all the datasets vertically
data_test = data_test[5000:] # the first dataset is included in twice
data_test.shape
# export data for MATLAB processing
print(savePath)
savePath_ = savePath + '\\test_data.mat'
print(savePath_)
savemat(savePath_, {'test_data':data_test})

In [ ]:
# Input test label dataset for all snr and stack them together
dataset = tf.data.Dataset.list_files('single_source_100snapshots_1000sample/*/test/test_label.*', shuffle=False)
for i in dataset.as_numpy_iterator():
    print(i)
data_set = []
for i in dataset.as_numpy_iterator():
    x_train = sio.loadmat(i)
    x_train = x_train['test_label']  # noisy sample covariance matrix
    data_set.append(x_train) #NOTE: put all the train_data of different SNR into one list
len(data_set) 
data_label = data_set[0]
for i in data_set:
    data_label = np.vstack((data_label,i)) # stack all the datasets vertically
data_label = data_label[5000:] # the first dataset is included in twice
data_label.shape

print(savePath)
savePath_ = savePath + '\\test_label.mat'
print(savePath_)
savemat(savePath_, {'test_label':data_label})

In [ ]:
encoded_data = autoencoder.encoder(data_test).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

# Save the denoised data for MATLAB processing
print(savePath)
savePath_ = savePath + '/denoised_data.mat'
print(savePath_)
savemat(savePath_, {'denoised_data':decoded_data})

ind = 200
plt.plot(data_test[ind],'b')
plt.plot(decoded_data[ind],'k')
plt.fill_between(np.arange(400),decoded_data[ind],data_test[ind],color='lightcoral')
plt.legend(["Input", "Reconstruction","Error"])
# plt.savefig('SNR_-30_autoencoder_compare.png')